# 微调

In [2]:
%matplotlib inline
import os
import torch
import torchvision
from torch import nn
from d2l import torch as d2l

## 下载数据集

In [3]:
#下载热狗数据集
d2l.DATA_HUB['hotdog'] = (d2l.DATA_URL + 'hotdog.zip',
                          'fba480ffa8aa7e0febbb511d181409f899b9baa5')

data_dir = d2l.download_extract('hotdog')

#ImageFolder是一个通用的数据加载器
train_imgs = torchvision.datasets.ImageFolder(os.path.join(data_dir, 'train'))
test_imgs = torchvision.datasets.ImageFolder(os.path.join(data_dir, 'test'))

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None))

## 显示图像

In [6]:
#下面显示了前8个正类样本图片和最后8张负类样本图片。 正如你所看到的，图像的大小和纵横比各有不同。
hotdogs = [train_imgs[i][0] for i in range(8)]
not_hotdogs = [train_imgs[-i - 1][0] for i in range(8)]
d2l.show_images(hotdogs + not_hotdogs, 2, 8, scale=1.4);

NameError: name 'train_imgs' is not defined

## 数据增广

In [18]:
#数据增广
#对RGB做三通道的均值和方差归一化
#如果用了BN，就不需要用这个了。
normalize = torchvision.transforms.Normalize([0.485, 0.456, 0.406],
                                             [0.229, 0.224, 0.225])

train_augs = torchvision.transforms.Compose([
    torchvision.transforms.RandomResizedCrop(224),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(), normalize])

test_augs = torchvision.transforms.Compose([
    torchvision.transforms.Resize(256),
    torchvision.transforms.CenterCrop(224),
    torchvision.transforms.ToTensor(), normalize])

## 定义初始化模型

In [5]:
#下载模型
#pretrained=True不仅把模型架构拿下来，而且用了在ImageNet上训练好的parameters
pretrained_net=torchvision.models.resnet18(pretrained=True)
#最后一层全连接层的参数
#只有这个不一样
pretrained_net.fc

Linear(in_features=512, out_features=1000, bias=True)

In [6]:
#随机初始化最后一层的参数
finetune_net=torchvision.models.resnet18(pretrained=True)
#finetune_net.fc.in_features保留原来的512,label是个二分类问题，输出通道数是2
finetune_net.fc=nn.Linear(finetune_net.fc.in_features,2)
#重新初始化最后一层全连接层的参数，其他层采用预训练模型的参数。
nn.init.xavier_uniform_(finetune_net.fc.weight)

Parameter containing:
tensor([[-0.0919, -0.1045, -0.0144,  ..., -0.0008,  0.0063, -0.0977],
        [ 0.0225,  0.0079, -0.0177,  ..., -0.0788, -0.0881,  0.0546]],
       requires_grad=True)

## 微调训练模型

In [7]:
def train_fine_tuning(net, learning_rate, batch_size=128, num_epochs=5,
                      param_group=True):
    train_iter = torch.utils.data.DataLoader(
        torchvision.datasets.ImageFolder(os.path.join(data_dir, 'train'),
                                         transform=train_augs),
        batch_size=batch_size, shuffle=True)
    test_iter = torch.utils.data.DataLoader(torchvision.datasets.ImageFolder(os.path.join(data_dir, 'test'),
                                         transform=test_augs),
        batch_size=batch_size)
    devices = d2l.try_all_gpus()
    loss = nn.CrossEntropyLoss(reduction="none")
    #param_group
    ## 如果param_group=True，输出层中的模型参数将使用十倍的学习率
    if param_group:
        params_1x = [param for name, param in net.named_parameters()
            if name not in ["fc.weight", "fc.bias"]]
        
        trainer = torch.optim.SGD([{
            'params': params_1x}, {
                'params': net.fc.parameters(),
                'lr': learning_rate * 10}], lr=learning_rate,
                                  weight_decay=0.001)
    else:
        trainer = torch.optim.SGD(net.parameters(), lr=learning_rate, weight_decay=0.001)
    d2l.train_ch13(net, train_iter, test_iter, loss, trainer, num_epochs,
                   devices)

In [8]:
#使用较小的学习率
train_fine_tuning(finetune_net, 5e-5)

NameError: name 'data_dir' is not defined

In [9]:
#为了进行比较，我们定义了一个相同的模型，但是将其所有模型参数初始化为随机值。 
#由于整个模型需要从头开始训练，因此我们需要使用更大的学习率。

#网络架构一样，只是不加载预训练的参数
scratch_net = torchvision.models.resnet18()
scratch_net.fc = nn.Linear(scratch_net.fc.in_features, 2)
train_fine_tuning(scratch_net, 5e-4, param_group=False)

NameError: name 'data_dir' is not defined

In [17]:
for param in finetune_net.parameters():
    param.requires_grad = False

weight = pretrained_net.fc.weight
#torch.split为分块，1为每块的大小，
hotdog_w1 = torch.split(weight.data, 1, dim=0)
#后面加上一个中括号，是为了将tuple变成tensor
hotdog_w = torch.split(weight.data, 1, dim=0)[0]
type(hotdog_w1),type(hotdog_w)

(tuple, torch.Tensor)

## 答疑

- 神经网络进行不同的目标检测，前面层的网络进行特征提取是通用的。越底层越通用。
- 源数据集和目标数据集要类似。
- 随机选择哪些层来迁移学习